In [7]:
import os
import caffe
import numpy as np

cnn_prototxt  = os.path.join('..', 'models', 'VGG_ILSVRC_19_layers_truncated_deploy.prototxt')
cnn_model     = os.path.join('..', 'models', 'VGG_ILSVRC_19_layers.caffemodel')

In [8]:
caffe.set_mode_gpu()
cnn_net  = caffe.Net(cnn_prototxt, cnn_model, caffe.TEST)
transformer = caffe.io.Transformer({'data': cnn_net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))
cnn_net.blobs['data'].reshape(50,3,224,224)

In [78]:
cnn_net.blobs['data'].data[...] = [transformer.preprocess('data', x) for x in np.random.rand(50, 224, 224, 3)]

In [79]:
%%time
out = cnn_net.forward()

CPU times: user 2.12 s, sys: 955 ms, total: 3.07 s
Wall time: 3.08 s


In [80]:
out['conv5_4'].shape

(50, 512, 14, 14)

In [89]:
import pandas as pd
pd.DataFrame([{'image': 'a', 'caption': 'asdf'}, {'image': 'b', 'caption': 'asdfas'}])

,caption,image
0,asdf,a
1,asdfas,b


In [74]:
images = pd.Series(df['image'].unique())
image_id_dict = pd.Series(np.array(images.index), index=images)
caption_image_id = df['image'].map(lambda x: image_id_dict[x]).values
cap = zip(df.caption.values, caption_image_id)
images[images.isin(['a'])]

0    a
dtype: object

In [35]:
dictionary_series

aasdf    2
asdf     4
absdf    3
dtype: int64